## Hidden Markov Model Introduction
  
Hidden Markov Model(HMM)이란 Markov Model 중 state 변화에 대하여 관찰자는 알지 못하고, 단지 결과만을 관측할 수 있는 Model 을 의미한다. 즉, state 변화가 hidden 되어 있다는 의미이다.  
아래는 위키피디아에서 제시한 예제인데 이를 보면 HMM에 대하여 이해하기 수월하다. 

> 관측자에게 보이지 않는 방이 있고, 그 안에 요정이 있다.방에는 항아리 $ X1,X2,X3,... $
    가 존재한다. 각각의 항아리에는 $ y_1,y_2,y_3,...$으로 이름이 붙은 공들이 섞여서 들어가 있고, 항아리의 공들이 섞인 상태는 알려져있다. 요정은 방 안의 항아리를 하나 고르고, 항아리 안에서 무작위로 공을 하나 꺼내어 컨베이어 벨트에 올려둔다. 관측자는 컨베이어 벨트 위의 공의 순서를 관측할 수는 있지만, 해당 공들이 뽑혔던 항아리의 순서는 알 수 없다. 요정이 공을 뽑을 항아리를 고를 때는 다음과 같은 특별한 규칙을 따른다 : n번째 공을 뽑기 위한 항아리는 무작위 숫자와 (n-1)번째 공을 뽑기 위해 선택한 항아리에만 영향을 받는다. 현재 항아리로부터 하나보다 더 전에 선택된 항아리는 현재 항아리의 선택에 직접적으로 영향을 주지 않는다. (https://ko.wikipedia.org/wiki/%EC%9D%80%EB%8B%89_%EB%A7%88%EB%A5%B4%EC%BD%94%ED%94%84_%EB%AA%A8%EB%8D%B8)

## HMM 구현 

HMM 구현을 위하여 새로운 예제를 하나더 제시하겠다. 이 또한 위키피디아에서 제시한 내용이다.
한 마을에 사람들이 있는데 그들의 건강 상태는 healthy, fever 두 상태가 있고 이에 대하여서는 오직 의사만이 결정할 수 있다. 이 의사는 환자에게 그들의 느낌을 물어서 건강 상태를 진단한다. 마을 사람들은 의사의 물음에 대하여 normal, dizzy, cold로 대답할 수 있다.   
의사는 건강 상태는 discrete MM 에 따른다고 믿고 있다. 여기서 상태는 healthy, fever 이고 이에 대하여서는 직접 알 방안은 없다. 단지 관찰값(환자의 대답)만이 주어질 뿐이다. (https://en.wikipedia.org/wiki/Viterbi_algorithm#Example)

이에 대하여 코드로 구현하여 보도록 하겠다. 코드는 위의 위키피디아와 아래 사이트를 참고하였다. 
(http://www.blackarbs.com/blog/introduction-hidden-markov-models-python-networkx-sklearn/2/9/2017)

###  State 정의 
우선 state 에 대하여 정의해보도록 하자. 여기서 state 는 환자의 상태, 즉 healthy 와 fever 이고, 시작 상태에 대한 확률, 즉 마을 주민들이 건강할 확률, 열이 있을 확률을 0.6 과 0.4 로 임의로 정했다. 

In [3]:
import numpy as np
import pandas as pd

%matplotlib inline

states = ['Healthy', 'Fever']
start_prob = [0.6, 0.4]
state_space = pd.Series(start_prob, index=states, name='states')
print(state_space)

start_p = {}
for i,st in enumerate(states):
    start_p[st] = start_prob[i]
print(start_p)

ImportError: No module named 'pandas'

s각 state 에 대한 transition probablity에 대한 matrix 는 아래와 같이 구성한다.   
H -> H = 0.7  
H -> F = 0.3  
F -> H = 0.4  
F -> F = 0.6  

In [4]:
# transition probability matrix 

trans_df = pd.DataFrame(columns=states, index=states)
trans_df.loc[states[0]] = [0.7, 0.3]
trans_df.loc[states[1]] = [0.4, 0.6]

print(trans_df)
trans_mat = trans_df.values

trans_p = {}
for i,row in enumerate(trans_mat):
    temp = {}
    for j,value in enumerate(row):
        temp[states[j]] = value
    trans_p[states[i]] = temp
print(trans_p)

NameError: name 'pd' is not defined

### Observation probability
observation probability 는 해당 상태일때, 관찰값이 나올 확률이다. 
즉, 여기서는 건강한 상태일 때 보통이라고 느낄 확률, 어지럽다고 느낄 확률, 춥다고 느낄 확률 등을 나타낸다. 
이 확률에 대한 matrix 는 state 갯수 X observation 이 된다. 

이 예제에서는 아래와 같이 정의하였다. 

In [5]:
observable_states = ['Normal', 'Cold', 'Dizzy']
emit_df = pd.DataFrame(columns=observable_states, index=states)
emit_df.loc[states[0]] = [0.5, 0.4, 0.1]
emit_df.loc[states[1]] = [0.1, 0.3, 0.6]

print(emit_df)

emit_mat = emit_df.values

emit_p = {}
for i,row in enumerate(emit_mat):
    temp = {}
    for j,value in enumerate(row):
        temp[observable_states[j]] = value
    emit_p[states[i]] = temp
print(emit_p)

NameError: name 'pd' is not defined

### Viterbi Algorithm
위의 예시에서 만약 의사가 삼일 동안 한명의 환자를 문진하였는데, 첫날은 normal, 두째날은 cold, 셋째날은 dizzy 가 그녀의 느낌이라면, 이러한 관찰 결과를 통하여 환자의 상태가 어떻게 변하였는지 알 수 있을까?  
즉, observation sequence 가 주어졌을 때 가장 확률이 높은 state transition sequence 는 어떻게 찾을 수 있을까?  
이를 찾는 알고리즘이 Viterbi Algorithm 이다. 

In [ ]:
def viterbi(obs, states, start_p, trans_p, emit_p):
    V = [{}]
    for st in states:
        V[0][st] = {"prob": start_p[st] * emit_p[st][obs[0]], "prev": None}
     # Run Viterbi when t > 0
    for t in range(1, len(obs)):
        V.append({})
        for st in states:
            max_tr_prob = max(V[t-1][prev_st]["prob"]*trans_p[prev_st][st] for prev_st in states)
            for prev_st in states:
                if V[t-1][prev_st]["prob"] * trans_p[prev_st][st] == max_tr_prob:
                    max_prob = max_tr_prob * emit_p[st][obs[t]]
                    V[t][st] = {"prob": max_prob, "prev": prev_st}
                    break
    for line in dptable(V):
        print(line)
    opt = []
    # The highest probability
    max_prob = max(value["prob"] for value in V[-1].values())
    previous = None
    # Get most probable state and its backtrack
    for st, data in V[-1].items():
        if data["prob"] == max_prob:
            opt.append(st)
            previous = st
            break
    # Follow the backtrack till the first observation
    for t in range(len(V) - 2, -1, -1):
        opt.insert(0, V[t + 1][previous]["prev"])
        previous = V[t + 1][previous]["prev"]
        
    print('The steps of states are ' + ' '.join(opt) + ' with highest probability of %s' % max_prob)

def dptable(V):
    # Print a table of steps from dictionary
    yield " ".join(("%12d" % i) for i in range(len(V)))
    for state in V[0]:
        yield "%.7s: " % state + " ".join("%.7s" % ("%f" % v[state]["prob"]) for v in V)

viterbi(['Normal', 'Cold', 'Dizzy'] , states, start_p, trans_p, emit_p)
